In [2]:
#Import statements
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from semantic_text_splitter import TextSplitter
from langchain_core.documents.base import Document
from tokenizers import Tokenizer

In [2]:
#How to load all the in once?
#Load in the PDFS
#loaders = [PyPDFLoader("C:/Users/elias/Downloads/WEF_The_Global_Cooperation_Barometer_2024.pdf")]

#loader = UnstructuredPDFLoader(file_path="C:/Users/elias/Downloads/WEF_The_Global_Cooperation_Barometer_2024.pdf")

# For one PDF this method works!:

# loader = PyPDFLoader("C:/Users/elias/Downloads/HPE ProLiant DX360 Gen10 Plus server-a50002577enw.pdf")
# data = loader.load()
# docs = []

# for file in loaders:
#     docs.extend(file.load())
    
# data[2].page_content

#Test to extends to multiple pdfs in once:
pdf_folder_path = "HPE Files"
documents = []
for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, file)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())

In [3]:
documents[1]

Document(metadata={'source': 'HPE Files\\HPE ProLiant ML350 Gen11-a50004308enw.pdf', 'page': 1}, page_content='QuickSpecs  HPE ProLiant ML350 Gen11 \nOverview \n \n \n \n \nPage 2 \n \n \n \nFront View – SFF chassis with optional Gen11 8SFF HDD Cage Kits shown (Tower mode) \n \n1. DisplayPort 1.1a 8 Optical drive bay \n2. USB 3.2 Gen1 port 9. Media bay filler panel \n3. iLO service port 10. Serial number/iLO information pull tab \n4. UID button/LED 11. Box3: Default drive cage for 8SFF* \n5. NIC status LED 12. Box2: Optional drive cage for 8SFF \n6. Health LED 13 Box1: Optional drive cage for 8SFF \n7. Power on/Standby button and system power LED   \nNotes: *Optional for 8SFF x4 U.3 drive kit or 12 EDSFF drive kit \n \n  \n')

In [4]:
#Split the pdfs
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# chunks = text_splitter.split_documents(documents)

#Split the pdfs using SemanticTextSplitter

max_tokens = 200
splitter = TextSplitter.from_tiktoken_model("gpt-3.5-turbo", max_tokens)

chunks = []
for document in documents:
    splitDocuments = []
    splitPageContents = splitter.chunks(document.page_content)
    i = 0
    for splitPageContent in splitPageContents:
        newMetadata = document.metadata
        newMetadata["partOfPage"] = i
        splitDocuments.append(Document(
            page_content=splitPageContent,
            metadata=newMetadata
        ))
        i += 1
    chunks.extend(splitDocuments)

In [5]:
chunks

[Document(metadata={'source': 'HPE Files\\HPE ProLiant ML350 Gen11-a50004308enw.pdf', 'page': 0, 'partOfPage': 0}, page_content='QuickSpecs  HPE ProLiant ML350 Gen11 \nOverview \n \n \n \n \nPage 1 \n \nHPE ProLiant ML350 Gen11  \nProvide most powerful and storage flexibility 2P tower server with rackable chassis design for various environments, and delivers \nexceptional compute performance, security, reliability, and expandability. Design to fulfill with wide range workloads for small \noffices, remote and branch offices of large enterprises, growing SMBs and data centers. HPE ProLiant ML350 Gen11 server is an \nexcellent choice to accelerate your growing business. \n \n \n \nFront View – LFF chassis with optional Gen11 4LFF HDD Cage Kits shown (Tower mode) \n1. DisplayPort 1.1a  8. Optical drive bay  \n2. USB 3.2 Gen1 port  9. Media bay filler panel  \n3. iLO service port  10. Serial number/iLO information pull tab'),
 Document(metadata={'source': 'HPE Files\\HPE ProLiant ML350 Gen1

In [6]:
#Load in a method to do the embeddings
!ollama pull nomic-embed-text

pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¸ pulling manifest â ´ pulling manifest â ¦ pulling manifest 
pulling 970aa74c0a90... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 274 MB                         
pulling c71d239df917... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  11 KB                         
pulling ce4a164fc046... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   17 B                         
pulling 31df23ea7daa... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [4]:
# Create new vector database with all the documents that are loaded
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag", persist_directory="DB"
)

TypeError: Chroma.from_documents() missing 1 required positional argument: 'documents'

In [ ]:
#Function to add something to existing Chroma DB so not entire DB has to be created.

In [15]:
#Retrieve DB
vector_db = Chroma(persist_directory="DB", embedding_function=OllamaEmbeddings(model="nomic-embed-text",show_progress=True))

In [16]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [17]:
#Load in LLM
!ollama pull mistral

pulling manifest â ™ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â ‡ pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ¹ pulling manifest â ¹ pulling manifest â ¼ pulling manifest â ¼ pulling manifest â ¦ pulling manifest â ¦ pulling manifest â § pulling manifest 
pulling ff82381e2bea... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 4.1 GB                         
pulling 43070e2d4e53... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  11 KB                         
pulling 491dfa501e59... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  801 B                         
pulling ed11eda7790d... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   30 B                         
pulling 42347cd80dc8... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  485 B                         
verifying sha256 digest 

In [18]:
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [19]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [20]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context: {context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [21]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
chain.invoke(input(""))

 What HPE products do you have the information about?


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.11s/it]


' Based on the provided context, there is no specific information about any Hewlett Packard Enterprise (HPE) products. Therefore, I cannot answer the question directly from the given context.'

In [11]:
chain.invoke(input(""))

 About which products do you have information


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.14s/it]


' Based on the provided context, there is no information about any specific products.'

In [22]:
chain.invoke(input(""))

 What do you have the information about?


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.13s/it]


' The provided text does not contain any information, so it cannot be determined what it is about.'